In [ ]:

from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()

SUBSCRIPTION = "<your-subscription-id>"
RESOURCE_GROUP = "<your-resource-group-name>"
WS_NAME = "<your-workspace-name>"

ml_client = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=WS_NAME,
)

In [ ]:

registered_model_name = "email_spam_model"

latest_model_version = max(
    [int(m.version) for m in ml_client.models.list(name=registered_model_name)]
)

print(latest_model_version)

In [ ]:

import uuid

online_endpoint_name = "email-spam-endpoint-" + str(uuid.uuid4())[:8]

In [ ]:

from azure.ai.ml.entities import ManagedOnlineEndpoint

endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is an online endpoint",
    auth_mode="key",
    tags={
        "training_dataset": "spam_email_data",
    },
)

In [ ]:

endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

In [ ]:
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(
    f'Endpoint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved'
)

In [ ]:
from azure.ai.ml.entities import ManagedOnlineDeployment

deploy_name = "email-spam-deployment-" + str(uuid.uuid4())[:8]

model = ml_client.models.get(name=registered_model_name, version=latest_model_version)

deployment = ManagedOnlineDeployment(
    name=deploy_name,
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_D2as_v4",
    instance_count=1,
)

In [ ]:

deployment = ml_client.online_deployments.begin_create_or_update(
    deployment
).result()

endpoint.traffic = {'deploy_traffic': 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

In [ ]:

endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(
    f"Name: {endpoint.name}\nStatus: {endpoint.provisioning_state}\nDescription: {endpoint.description}"
)

In [ ]:

print(endpoint.traffic)

print(endpoint.scoring_uri)

Check Endpoint

In [ ]:
import os

deploy_dir = "./deploy"
os.makedirs(deploy_dir, exist_ok=True)

In [ ]:
%%writefile {deploy_dir}/sample-request.json
{
    "input_data": {"Message": "Hello, how are you?"}
}

In [ ]:

ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name=deploy_name,
    request_file="./deploy/sample-request.json",
)